# Spark SQL Flexible Agent, going beyond SQL queries
The classical ZeroShotAgent always forces to use a tool which makes sense when the tool can finish the work better than AI, however, there are many cases, AI is better than tools, esp. when generation is needed.

One example, when user uses Spark, sometimes, we want the model to give us suggestions of how to write some queries to assist our work instead of doing the query for us directly. (Letting the AI doing the query is a valid case but it's not the only case) The power of LLM is generation. It's able to provide more creative and complex answer, which includes providing suggestions to human.

In this notebook, we present how to enable AI's creativity in the classical MRKL framework through a freestyle tool.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
schema = "langchain_example"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema}")
spark.sql(f"USE {schema}")
csv_file_path = "../toolkits/examples/titanic.csv"
table = "titanic"
spark.read.csv(csv_file_path, header=True, inferSchema=True).write.mode('overwrite').saveAsTable(table)
spark.table(table).show()

23/05/13 21:51:05 WARN Utils: Your hostname, Mikes-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.68.115 instead (on interface en1)
23/05/13 21:51:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/13 21:51:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/13 21:51:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/13 21:51:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/skcoirz/GitHub/spark_langchain/docs/modules/agents/agents/toolkits/examples/titanic.csv.